In [ ]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from pprint import pprint

In [ ]:
username = 'm&e.unicef'
password = 'Unicef@MoH_DHI9'

In [ ]:
CONFIG =  {
    'api_entrypoint': 'https://hmis.health.go.ug/api/'
}

In [ ]:
def get_dhis_index_table(auth, page_size=70000, table='dataElements'):
    ENTRY = CONFIG.get('api_entrypoint')
    response = requests.get(ENTRY + f'{table}?pageSize={page_size}', auth=auth)
    df = pd.DataFrame(response.json().get(table))
    return df

In [ ]:
def set_name_from_index(df, column_name, index_table=None, fetch_index=False, auth=None):
    if fetch_index:
        assert auth is not None, 'Cannot retrieve the index tables without authentication object'
        index_table = get_dhis_index_table(auth, page_size=10000, table=column_name+'s')
    else:
        assert index_table is not None, 'Set fetch index or pass index table'
    if column_name == 'organisationUnit':
        column_name = 'orgUnit'
        index_table.rename(columns={'organisationUnit': 'orgUnit'})
    df[column_name] = df[column_name].apply(lambda x: index_table[index_table['id'].str.contains(x)]['displayName'].values[0])
    return df

In [ ]:
auth = HTTPBasicAuth(username, password)

In [ ]:
df = pd.DataFrame(requests.get('https://hmis.health.go.ug/api/dataValueSets?dataSet=IAtaRO1bTAO&orgUnit=j4zrX8QSkrA&period=2019', auth=auth).json().get('dataValues'))

In [ ]:
df.head()

In [ ]:
org_unit_index = get_dhis_index_table(auth, page_size=10000, table='organisationUnits')

In [ ]:
data_elements = pd.DataFrame(requests.get('https://hmis.health.go.ug/api/dataElements?pageSize=10000', auth=auth).json().get('dataElements'))

In [ ]:
data_elements.head()

In [ ]:
data_elements[data_elements['displayName'].str.contains('105-AN01a')].values[-1]

In [ ]:
data_sets = [x.get('dataSet').get('id') for x in requests.get('https://hmis.health.go.ug/api/dataElements/Q9nSogNmKPt?fields=dataSetElements', auth=auth).json().get('dataSetElements')]

In [ ]:
data_sets

In [ ]:
x = data_sets[0]

In [ ]:
data_set = requests.get(f'https://hmis.health.go.ug/api/dataSets/{x}', auth=auth).json()

In [ ]:
pprint(data_set)

In [ ]:
org_units  = [x.get('id') for x in requests.get(f'https://hmis.health.go.ug/api/dataSets/{x}', auth=auth).json().get('organisationUnits')]

In [ ]:
x

In [ ]:
org_units_string = '&orgUnit='.join(org_units[50:100])

In [ ]:
org_units_string

In [ ]:
df = pd.DataFrame(requests.get(f'https://hmis.health.go.ug/api/dataValueSets?dataSet={x}&{org_units_string}&lastUpdatedDuration=100d&dataElementGroup=W1NQYn9zeT0&dataElementGroup=vPvLSIPrfnb', auth=auth).json().get('dataValues'))

In [ ]:
df.head()

In [ ]:
set_name_from_index(df, 'dataElement', auth=auth, fetch_index=True)

In [ ]:
set_name_from_index(df, 'orgUnit', index_table=org_unit_index)